In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd

dataframe: pd.DataFrame = pd.read_csv('../resources/sampled_reviews_with_output_multicall_phi3_medium.csv')
dataframe.head()

,Unnamed: 0,review,sentiment,entities,json,progressive_index,output
0,434,"1st watched 2/9/2008, 4 out of 10(Dir-J.S. Car...",negative,"[{'label': 'ORG', 'value': 'qwest'}, {'label':...","{""review"": ""1st watched 2/9/2008, 4 out of 10(...",435,"```json\n{\n ""sentiment"": ""negative"",\n ""re..."
1,3278,Following on directly from the last episode of...,positive,"[{'label': 'PERSON', 'value': 'Hacker'}, {'lab...","{""review"": ""Following on directly from the las...",3279,"```json\n{\n ""sentiment"": ""positive"",\n ""re..."
2,967,But at least this movie got what it deserved -...,negative,"[{'label': 'PERSON', 'value': 'Mike'}, {'label...","{""review"": ""But at least this movie got what i...",968,"```json\n{\n ""sentiment"": ""NEGATIVE"",\n ""re..."
3,1982,A few buddies and myself have the strange hobb...,negative,"[{'label': 'PERSON', 'value': 'Larry Buchanan'...","{""review"": ""A few buddies and myself have the ...",1983,"Based on the review text provided, here is th..."
4,2902,On a routine mission in Iraq a group of Delta ...,positive,"[{'label': 'ORG', 'value': 'Delta'}, {'label':...","{""review"": ""On a routine mission in Iraq a gro...",2903,"```json\n{\n ""sentiment"": ""positive"",\n ""re..."


In [8]:
dataframe[dataframe.output == "$$$"]

,Unnamed: 0,review,sentiment,entities,json,progressive_index,output


In [3]:
cleaned: pd.DataFrame = dataframe.apply(lambda row: row["output"] if row["output"] and row["output"].strip() != "" else "$$$", axis=1)
cleaned.head()

,review,sentiment,entities,json,progressive_index
0,With No Dead Heroes you get stupid lines like ...,negative,"[{'label': 'PERSON', 'value': 'William Sanders...","{""review"": ""With No Dead Heroes you get stupid...",1
1,I thought maybe... maybe this could be good. A...,negative,"[{'label': 'PERSON', 'value': 'Jeffery Combs'}...","{""review"": ""I thought maybe... maybe this coul...",2
2,An elite American military team which of cours...,negative,"[{'label': 'ORG', 'value': 'Spetznaz'}, {'labe...","{""review"": ""An elite American military team wh...",3
3,Ridiculous horror film about a wealthy man (Jo...,negative,"[{'label': 'PERSON', 'value': 'John Carradine'...","{""review"": ""Ridiculous horror film about a wea...",4
4,"Well, if you are one of those Katana's film-nu...",positive,"[{'label': 'PERSON', 'value': 'Katana'}, {'lab...","{""review"": ""Well, if you are one of those Kata...",5


In [5]:
cleaned = cleaned.sample(n=1000)
cleaned.to_csv('../resources/IMDB Dataset Sampled.csv')